In [4]:
from IPython.display import HTML
from geopy.distance import geodesic

import pandas as pd

In [5]:
# latitude and longtitude of each place
# pos_list = [
#     [43.06417, 141.34694, "Sapporo"], 
#     [38.26889, 140.87194, "Sendai"], 
#     [36.59444, 136.62556, "Kanazawa"], 
#     [35.68944, 139.69167, "Tokyo"], 
#     [35.18028, 136.90667, "Nagoya"]
# ]

all_places = [
    [40.6643, -73.9385, "New York"],
    [38.9041, -77.0171, "Washington"], 
    [42.332, -71.0202, "Boston"],
    [45.4208, -75.6945, "Otawwa"],
    [43.7166, -79.3407, "Tront"], 
    [46.8127, -71.2199, "Quebec"],
    [45.5088, -73.5878, "Montreal"]
]
departure =  all_places[0]
pos_list = all_places[1:len(all_places)]
point_num = len(pos_list)


# agents
agents = ["Agent1", "Agent2"]
agent_num = len(agents)

num_clusters = int(len(pos_list)/agent_num)
load_capa = agent_num

In [6]:
# calculate distance from latitude and longitude
def calc_distance(coord1: list, coord2: list) -> float:
    return geodesic(coord1, coord2).kilometers

In [7]:
def cluster_distance(pos_list, departure, num_clusters, load_capa):
    # calculate the distance between each two places
    distances = []
    places = []
    for place in pos_list:
        distance = calc_distance(departure[:2], place[:2])
        places.append(place[2])
        distances.append(distance)

    dist_df = pd.DataFrame(data={
        "place": places,
        "distance": distances,
    })
    dist_df = dist_df.sort_values("distance").reset_index() # sort by distance
    
    # create clusters
    clusters = {}
    for i in range(num_clusters):
        cluster = []
        for j in range(load_capa):
            cluster.append([dist_df.iloc[load_capa*i + j]['index'],
                            dist_df.iloc[load_capa*i + j]['place'], 
                            float(dist_df.iloc[load_capa*i + j]['distance'])])
        clusters[i] = cluster

    return dist_df, clusters

In [ ]:
import folium

# function to show results on a map
def visualize(df, clusters):
    # map size
    f = folium.Figure(width=600, height=600)

    # create the map with center latitude and longtitude
    center_lat = 40.000
    center_log = -75.000
    map = folium.Map(location=[center_lat,center_log], zoom_start=5)

    # put pins on the map
    folium.Marker([departure[0], departure[1]], popup=departure[2]).add_to(map)
    for pos in pos_list:
        folium.Marker([pos[0], pos[1]], popup=pos[2]).add_to(map)

    # the color of the line
    color = ["#0000ff", "#ff0000", "#008000"]

    # the order of the places to visit
    order_list = []
    for i in range(agent_num):
        order = [0]
        for j in range(point_num):
            if df[i][j] == 1:
                order.append(j+1)
        order.append(0)
        order_list.append(order)

    # draw a line between two places depending on the order
    for a in range(agent_num):
        for p in range(len(clusters)):
            folium.PolyLine(locations=[all_places[order_list[a][p]][:2], all_places[order_list[a][p+1]][:2]],
                            weight=3,color=color[a]).add_to(map)
        # return to the departure
        folium.PolyLine(locations=[all_places[order_list[a][p+1]][:2], all_places[order_list[a][0]][:2]],
                            weight=3,color=color[a]).add_to(map)
    f.add_child(map)

    return f

In [9]:
from pyqubo import solve_qubo, Array, Placeholder

# create spins
x = Array.create('x', shape=(agent_num, point_num), vartype='BINARY')

In [10]:
# caluculate distance between two places
distance_list = []
for i in range(point_num):
    distance = []
    for j in range(point_num):
        distance.append(calc_distance(pos_list[i][:2], pos_list[j][:2]))
    distance_list.append(distance)

print(distance_list)

[[0.0, 634.0821887798886, 732.0274016495632, 568.7255062906598, 997.4926303166757, 786.1934563781998], [634.0821887798886, 0.0, 508.6807498730002, 695.090101048841, 498.1628749928413, 408.7435706161688], [732.0274016495632, 508.6807498730002, 0.0, 346.0008104303673, 378.83798927281117, 165.04342318958064], [568.7255062906598, 695.090101048841, 346.0008104303673, 0.0, 723.7832857218843, 498.0295270964522], [997.4926303166757, 498.1628749928413, 378.83798927281117, 723.7832857218843, 0.0, 233.33952385374388], [786.1934563781998, 408.7435706161688, 165.04342318958064, 498.0295270964522, 233.33952385374388, 0.0]]


In [11]:
dist_df, clusters = cluster_distance(pos_list, departure, num_clusters, load_capa)
print(clusters[0])

[[np.int64(1), 'Boston', 306.045644580377], [np.int64(0), 'Washington', 328.2061794645842]]


In [12]:
# minimize the total distance
H1 = 0
total_distance = {}
# roup of each agent
for a in range(agent_num):
    distance = 0
    for c in range(len(clusters) - 1):
        if c == 0:
            for i in range(load_capa):
                point_idx = clusters[c][i][0] # the place to visit next to the departure
                distance +=  clusters[c][i][2] * x[a, point_idx]
        for i in range(load_capa):
            for j in range(load_capa):
                point1_idx = clusters[c][i][0]
                point2_idx = clusters[c + 1][j][0]
                H1 += distance_list[point1_idx][point2_idx] * x[a, point1_idx] * x[a, point2_idx]
                if c == len(clusters) - 1:
                    distance +=  clusters[c + 1][i][2] * x[a, point2_idx] # from the last place to the departure
    total_distance[a] = distance
    H1 += distance

In [13]:
# Each place is visited only once
H2 = 0
for p in range(point_num):
    H2_1 = 0
    for a in range(agent_num):
        H2_1 += x[a, p]
    H2 += (H2_1 - 1)**2

In [14]:
# Each agent visits one place from only one group
H3 = 0
for a in range(agent_num):
    for c in range(len(clusters)):
        H3_1 = 0
        for i in range(len(clusters[c])):
            H3_1 += x[a, clusters[c][i][0]]
        H3 += (H3_1 - 1)**2

In [15]:
# each agent visits one place at a time
H4 = 0
for a in range(agent_num):
    H4_1 = 0
    for p in range(point_num):
        H4_1 += x[a, p]
    H4 += (H4_1 - 1)**2

In [16]:
# each agents visit the same numbers of the places
expected_num_places = len(clusters)
H5 = 0
for a in range(agent_num):
    H5_1 = 0
    for p in range(point_num):
        H5_1 += x[a, p]
    H5 += (H5_1 - expected_num_places)**2

In [17]:
# each agent's distance 
average_distance = H1 / agent_num
H6 = 0
for a in range(agent_num):
    H6 += (total_distance[a] - average_distance)**2

In [18]:
H = Placeholder('param_1') * H1 + Placeholder('param_2') * H2 + Placeholder('param_3') * H3 + \
    Placeholder('param_4') * H4 + Placeholder('param_5') * H5 + Placeholder('param_6') * H6

model = H.compile()
feed_dict = {'param_1': 0.001, 'param_2': 4, 'param_3': 1, 'param_4': 1, 'param_5': 3, 'param_6': 0.001}
qubo, offset = model.to_qubo(feed_dict=feed_dict)

In [19]:
# D-Wave
import dimod
import neal

# setting for D-Wave
bqm = dimod.BQM(qubo, 'BINARY')

In [20]:
# quantum aneealing

# token = xxxxx
# endpoint = xxxxx

# sampling
# dw_sampler = DWaveSampler(solver='Advantage2_prototype2.5', token=token, endpoint=endpoint)
# sampler = EmbeddingComposite(dw_sampler)
# response = sampler.sample(bqm, num_reads=100)

In [21]:
# simulated annealing
sampler = neal.SimulatedAnnealingSampler()
response = sampler.sample_qubo(qubo)
solution = response.first.sample # the best solution

energy = int(response.first.energy)

In [22]:
# Print the best solution
response.to_pandas_dataframe().head()

,x[0][0],x[0][0] * x[0][2],x[0][0] * x[0][5],x[0][1],x[0][1] * x[0][2],x[0][1] * x[0][5],x[0][2],x[0][2] * x[0][3],x[0][2] * x[0][4],x[0][3],...,x[1][2],x[1][2] * x[1][3],x[1][2] * x[1][4],x[1][3],x[1][3] * x[1][5],x[1][4],x[1][4] * x[1][5],x[1][5],energy,num_occurrences
0,0,0,0,1,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,-89.120204,1


In [23]:
# comfirm the result meets the constraints
decoded_samples = model.decode_sampleset(response, feed_dict=feed_dict)

for sample in decoded_samples:
    print(sample.energy)
    print(sample.constraints(only_broken=True))

-3.120203522777956
{}


In [24]:
# result of the d-wave quantum annealing
# decoded_sampleset = model.decode_sampleset(response, feed_dict=feed_dict)
# best_sample = min(decoded_sampleset, key=lambda s: s.energy)
# # print(best_sample)

In [25]:
result_list = []
for p in range(point_num):
    result = []
    for j in range(point_num):
        if j == dist_df.iloc[p]['index']:
            for i in range(agent_num):
                result.append(solution[f'x[{i}][{j}]'])
    result_list.append(result)

In [26]:
# print the result
df = pd.DataFrame(result_list)
df_place_idx = df.copy()
df_place_idx.index = dist_df['place']
print(df_place_idx)

            0  1
place           
Boston      1  0
Washington  0  1
Montreal    1  0
Otawwa      0  1
Tront       0  1
Quebec      1  0


In [27]:
#  visualize the result
visualize(df, clusters)

0
1
2
0
1
2
